In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt
import time


In [2]:
file_path = 'Resources\sim_data.csv'
sim_df = pd.read_csv(file_path)
sim_df.head()

,race_name,date,full_name,finish_position,grid_position,avg_humidity,avg_air_pressure,rainfall,avg_airtemp,downforce_level,start_tyre,end_tyre,num_stops,q1,q2,q3,Unnamed: 16
0,70th Anniversary Grand Prix,8/9/2020,Valtteri Bottas,3.0,1,58.556034,1000.950862,f,25.287069,High,MEDIUM,HARD,3.0,01:26.7,01:25.8,01:25.2,NaN
1,70th Anniversary Grand Prix,8/9/2020,Lewis Hamilton,2.0,2,58.556034,1000.950862,f,25.287069,High,MEDIUM,HARD,3.0,01:26.8,01:26.3,01:25.2,NaN
2,70th Anniversary Grand Prix,8/9/2020,Nico Hulkenberg,7.0,3,58.556034,1000.950862,f,25.287069,High,MEDIUM,SOFT,4.0,01:27.3,01:26.3,01:26.1,NaN
3,70th Anniversary Grand Prix,8/9/2020,Max Verstappen,1.0,4,58.556034,1000.950862,f,25.287069,High,HARD,HARD,3.0,01:27.2,01:26.8,01:26.2,NaN
4,70th Anniversary Grand Prix,8/9/2020,Daniel Ricciardo,14.0,5,58.556034,1000.950862,f,25.287069,High,MEDIUM,MEDIUM,4.0,01:27.4,01:26.6,01:26.3,NaN


In [3]:
# check columns and rows that are not null
sim_df.count()

race_name           1136
date                1136
full_name           1136
finish_position      989
grid_position       1136
avg_humidity        1136
avg_air_pressure    1136
rainfall            1136
avg_airtemp         1136
downforce_level      716
start_tyre          1117
end_tyre            1117
num_stops           1117
q1                  1120
q2                   841
q3                   555
Unnamed: 16            0
dtype: int64

In [4]:
# check columns and rows that contain nulls
sim_df.isnull().sum()

race_name              0
date                   0
full_name              0
finish_position      147
grid_position          0
avg_humidity           0
avg_air_pressure       0
rainfall               0
avg_airtemp            0
downforce_level      420
start_tyre            19
end_tyre              19
num_stops             19
q1                    16
q2                   295
q3                   581
Unnamed: 16         1136
dtype: int64

In [5]:
# column data types
sim_df.dtypes

race_name            object
date                 object
full_name            object
finish_position     float64
grid_position         int64
avg_humidity        float64
avg_air_pressure    float64
rainfall             object
avg_airtemp         float64
downforce_level      object
start_tyre           object
end_tyre             object
num_stops           float64
q1                   object
q2                   object
q3                   object
Unnamed: 16         float64
dtype: object

In [6]:
#  ADDING REGEX HERE to replace second colon in qualifying time with decimal so datetime conversion below will run correctly

In [7]:
# sim_df['q1'] = pd.to_datetime(sim_df['q1'], format='%M:%S.%f')
# sim_df['q2'] = pd.to_datetime(sim_df['q2'], format='%M:%S.%f')
# sim_df['q3'] = pd.to_datetime(sim_df['q3'], format='%M:%S.%f')
# sim_df.head()

ValueError: time data '0:01:36' does not match format '%M:%S.%f' (match)

In [ ]:
sim_df['finish_position'].head()

In [ ]:
# If a driver does not finish (DNF) a race they are represented as blanks in finish_position column. 
# To reperesent the DNF cells, this replaces NaN with 20, as in 20th place

sim_df['finish_position'] = sim_df['finish_position'].fillna(20)
sim_df['finish_position'].head(40)

In [ ]:
sim_df.head()

In [ ]:
sim_df['qualifying_times'] = sim_df[['q1', 'q2', 'q3']].min(axis=1)

In [ ]:
sim_df.head()

In [ ]:
# drop columns not needed
sim_df = sim_df.drop(['downforce_level', 'q1', 'q2', 'q3'], axis=1)
sim_df.head()

In [ ]:
sim_df = sim_df.drop('Unnamed: 16', axis=1)
sim_df.head()

In [ ]:
sim_df['rainfall'] = np.where(sim_df['rainfall'] == 'False' , 0 ,1)

In [ ]:
sim_df.head()

In [ ]:
sim_df['start_tyre'].unique()

In [ ]:
# assign tire types to integers

d = {np.nan:0, 'SOFT':1,'MEDIUM':2,'HARD':3, 'INTERMEDIATE':4, 'WET':5}

sim_df['start_tyre'] = sim_df['start_tyre'].map(d).astype(int)
sim_df.head(10)

In [ ]:
# assign tire types to integers

d = {np.nan:0, 'SOFT':1,'MEDIUM':2,'HARD':3, 'INTERMEDIATE':4, 'WET':5}

sim_df['end_tyre'] = sim_df['end_tyre'].map(d).astype(int)
sim_df.head(10)

In [ ]:
sim_df.dtypes

In [ ]:
#sim_df['qualifying_times'].dt.hour * 3600 +
sim_df['qt_seconds'] = sim_df['qualifying_times'].dt.hour * 3600 + sim_df['qualifying_times'].dt.minute * 60 + sim_df['qualifying_times'].dt.second
sim_df.head()

In [ ]:
# ADD VISUALIZATIONS - histograms?

In [ ]:
# avg_driver_finish = sim_df.groupby(["full_name"]).mean()["finish_position"]
# avg_driver_finish

In [ ]:
# avg_driver_qual = sim_df.groupby(["full_name"]).mean()["qualifying_time"]
# avg_driver_qual